# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [1]:
# from __future__ import division
# import graphlab
import math
import string
import turicreate as tc
import numpy as np
import pandas as pd

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = tc.SFrame('amazon_baby.gl/')

Now, let us see a preview of what the dataset looks like.

In [3]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [4]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'review': 'A favorite in our house!',
 'rating': 5.0}

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [5]:
def remove_punctuation(text):
    import string
#     return text.translate(None, string.punctuation) 
    return text.translate(string.punctuation) 

review_without_punctuation = products['review'].apply(remove_punctuation)
products['word_count'] = tc.text_analytics.count_words(review_without_punctuation)

In [6]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'review': 'A favorite in our house!',
 'rating': 5.0,
 'word_count': {'our': 1, 'in': 1, 'house': 1, 'favorite': 1, 'a': 1}}

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [7]:
products[269]['word_count']

{'our': 1, 'in': 1, 'house': 1, 'favorite': 1, 'a': 1}

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [8]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [9]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1, 'moist':1, 'osocozy': 1, 'keps': ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1,'comfortable': 1, 'bed': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1, 'clever': 1,'little': 1, 'loves': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1, 'many': 1,'headaches': 1, 'soo' ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1, 'or': 1,'break': 1, 'trying': 1, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1, 'barnes': 1,'at': 1, 'is': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1, 'because':1, 'questions': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1, 'and': 1,'changes': 1, 'the': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1, 'pages': 1,'out': 1, 'run': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1,'recommend': 1, 'high ...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [10]:
train_data, test_data = products.random_split(.8, seed=1)
print(len(train_data))
print(len(test_data))

133416
33336


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [11]:
sentiment_model = tc.logistic_classifier.create(train_data,
                                                      target = 'sentiment',
                                                      features=['word_count'],
                                                      validation_set=None)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 13       | 1.000000  | 2.757545     | 0.942998          |

| 10        | 18       | 1.000000  | 3.586206     | 0.963468          |

+-----------+----------+-----------+--------------+-------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [12]:
sentiment_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 57446
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 57445

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 10
Solver status                  : Completed (Iteration limit reached).
Training time (sec)            : 3.6603

Settings
--------
Log-likelihood                 : 14186.1869

Highest Positive Coefficients
-----------------------------
word_count[arghhhhhh]          : 11.5831
word_count[chinaseveral]       : 8.0591
word_count[finallt]            : 7.3705
word_count[gfis]               : 7.2048
word_count[negatories]         : 7.1795

Lowest Negative Coefficients
----------------------------
word_count[5gal]               : 

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [13]:
weights = sentiment_model.coefficients
weights.column_names()

['name', 'index', 'class', 'value', 'stderr']

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [49]:
num_positive_weights = sum(weights['value'] >= 0)
num_negative_weights = sum(weights['value'] < 0)

print("Number of positive weights: %s " % num_positive_weights)
print("Number of negative weights: %s " % num_negative_weights)

Number of positive weights: 40814 
Number of negative weights: 16632 


**Quiz Question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [15]:
sample_test_data = test_data[10:13]
print(sample_test_data['rating'])
sample_test_data

[5.0, 2.0, 1.0]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'again': 1, 'book': 1,'same': 1, 'receive': 1, ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'peeling': 1, '5': 1,'about': 1, 'for': 1, ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,"{'on': 1, 'waste': 1,'wall': 1, 'frustrati ...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [16]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [17]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [18]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print(scores)

[3.643812624448069, -2.2473752116922388, -6.06983358459325]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [19]:
def predicting_sentiment(scores):
    yhat = [-1 if s <= 0 else 1 for s in scores]
    return yhat

predicting_sentiment(scores)

[1, -1, -1]

Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [20]:
print("Class predictions according to GraphLab Create:")
print(sentiment_model.predict(sample_test_data))

Class predictions according to GraphLab Create:
[1, -1, -1]


**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [21]:
def probability_predictions(scores):
    yhat = [1/(1+np.exp(-s)) for s in scores]
    return yhat
                
probability_predictions(scores)

[0.9745140750321416, 0.095576114342366944, 0.0023062268805699201]

**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [22]:
print("Class predictions according to GraphLab Create:")
print(sentiment_model.predict(sample_test_data, output_type='probability'))

Class predictions according to GraphLab Create:
[0.9745140750321416, 0.09557611434236694, 0.0023062268805699193]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [23]:
prob = sentiment_model.predict(test_data, output_type='probability')
prob = prob.is_topk(topk=50, reverse=False)
df = pd.DataFrame()
df['Name'] = test_data['name']
df['Probability'] = prob
df.sort_values(by='Probability', inplace=True, ascending=False)
df.head(50)

,Name,Probability
10170,We Sell Mats 36 Sq Ft Alphabet and Number Floo...,1
16502,"Dream On Me / Mia Moda Atmosferra Stroller, Nero",1
21531,Roan Rocco Classic Pram Stroller 2-in-1 with B...,1
260,Munchkin Mozart Magic Cube,1
21606,"Quinny 2012 Buzz Stroller, Rebel Red",1
7501,"Moby Wrap Original 100% Cotton Baby Carrier, Red",1
14482,"Simple Wishes Hands-Free Breastpump Bra, Pink,...",1
30297,Britax Boulevard 70-G3 Convertible Car Seat Se...,1
30321,Ubooly iPhone and iPod Interactive Pet,1
31271,Phil &amp; Teds Navigator Buggy Golden Kiwi Fr...,1


**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [24]:
prob = sentiment_model.predict(test_data, output_type='probability')
prob = prob.is_topk(topk=20, reverse=True)
df = pd.DataFrame()
df['Name'] = test_data['name']
df['Probability'] = prob
df.sort_values(by='Probability', inplace=True, ascending=False)
df.head(20)

,Name,Probability
30538,Mother's Lounge 5 Piece Carseat Canopy Whole C...,1
2931,Fisher-Price Ocean Wonders Aquarium Bouncer,1
1469,Fisher-Price Royal Potty,1
17034,Snuza Portable Baby Movement Monitor,1
30373,Samsung SEW-3037W Wireless Pan Tilt Video Baby...,1
30380,Samsung SEW-3037W Wireless Pan Tilt Video Baby...,1
26924,"Munchkin Nursery Projector and Sound System, W...",1
32403,MyLine Eco baby play mat-Animal Friend/Train ABC,1
1942,Philips AVENT Newborn Starter Set,1
21700,Levana Safe N'See Digital Video Baby Monitor w...,1


**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [25]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    scores = np.array(model.predict(data, output_type='margin'))
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    yhat = np.array(predicting_sentiment(scores))
    correct = [i for i, j in zip(yhat, true_labels.to_numpy()) if i == j]
    correct = np.array(correct)

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = len(correct)/len(yhat)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [26]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.925755939524838

In [27]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

0.9634676500569647

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [28]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [29]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [30]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [31]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [32]:
print(train_data[0]['word_count'])

{'recommend': 1, 'moist': 1, 'osocozy': 1, 'keps': 1, 'leak': 1, 'holder': 1, 'was': 1, 'now': 1, 'wipe': 1, 'my': 2, 'disappointed': 1, 'wise': 1, 'and': 3, 'planet': 1, 'it': 3, 'love': 1, 'early': 1, 'not': 2, 'highly': 1, 'wipes': 1, 'bags': 1, 'came': 1, 'does': 1, 'i': 1}


Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [33]:
print(train_data[0]['word_count_subset'])

{'disappointed': 1, 'love': 1}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [34]:
simple_model = tc.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.235944     | 0.867242          |

| 6         | 7        | 0.274738     | 0.867242          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 21
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 20

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.2828

Settings
--------
Log-likelihood                 : 44164.7122

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]       : 1.6816
word_count_subset[perfect]     : 1.5109
word_count_subset[love]        : 1.3581
(intercept)                    : 1.2933
word_count_subset[easy]        : 1.1846

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed] : -2.3571
wo

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [35]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8702303815694744

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [36]:
simple_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.2932808921548138,0.01214597839135102
word_count_subset,disappointed,1,-2.357103739416159,0.05006337714164177
word_count_subset,love,1,1.3580674001759903,0.030174584532469358
word_count_subset,would,1,-0.36339953884017884,0.012733705968756026
word_count_subset,little,1,0.5150234016435916,0.02147647444204097
word_count_subset,loves,1,1.68161202474526,0.04841536802265394
word_count_subset,product,1,-0.32668190175120987,0.015215122005224312
word_count_subset,well,1,0.5306735620456048,0.02125574017349695
word_count_subset,great,1,0.9441757614322582,0.020744752047621096
word_count_subset,easy,1,1.1845752461137058,0.028955683944966275


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [37]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+--------------+-------+----------------------+
|        name       |    index     | class |        value         |
+-------------------+--------------+-------+----------------------+
| word_count_subset |    loves     |   1   |   1.68161202474526   |
| word_count_subset |   perfect    |   1   |  1.510929148516271   |
| word_count_subset |     love     |   1   |  1.3580674001759903  |
|    (intercept)    |     None     |   1   |  1.2932808921548138  |
| word_count_subset |     easy     |   1   |  1.1845752461137058  |
| word_count_subset |    great     |   1   |  0.9441757614322582  |
| word_count_subset |     well     |   1   |  0.5306735620456048  |
| word_count_subset |    little    |   1   |  0.5150234016435916  |
| word_count_subset |     able     |   1   | 0.19353097465848976  |
| word_count_subset |     old      |   1   | 0.08269162076978379  |
| word_count_subset |     car      |   1   | 0.05648814949816574  |
| word_count_subset |     less     |   1   | -0.

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [38]:
len(simple_model.coefficients[simple_model.coefficients['value'] >= 0])-1

10

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [52]:
simple_model.coefficients[simple_model.coefficients['value'] >= 0]

name,index,class,value,stderr
(intercept),None,1,1.2932808921548138,0.01214597839135102
word_count_subset,love,1,1.3580674001759903,0.030174584532469358
word_count_subset,little,1,0.5150234016435916,0.02147647444204097
word_count_subset,loves,1,1.68161202474526,0.04841536802265394
word_count_subset,well,1,0.5306735620456048,0.02125574017349695
word_count_subset,great,1,0.9441757614322582,0.020744752047621096
word_count_subset,easy,1,1.1845752461137058,0.028955683944966275
word_count_subset,able,1,0.19353097465848976,0.033763017755208624
word_count_subset,perfect,1,1.510929148516271,0.04936790911609711
word_count_subset,old,1,0.08269162076978379,0.01944270201933346


In [53]:
sentiment_model.coefficients[sentiment_model.coefficients['value'] >= 0]

name,index,class,value,stderr
(intercept),None,1,0.5587753188182517,None
word_count,recommend,1,0.3176005738337419,None
word_count,osocozy,1,0.1353374130694127,None
word_count,keps,1,2.6667826468261597,None
word_count,holder,1,0.09260523307343804,None
word_count,now,1,0.056486739663825704,None
word_count,wipe,1,0.15517628599426028,None
word_count,my,1,0.08162805643958625,None
word_count,and,1,0.030093548568454673,None
word_count,it,1,0.012370194486988551,None


# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [41]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

0.9634676500569647

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [42]:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])

0.8672423097679439

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [43]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.925755939524838

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [44]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8702303815694744

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [45]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print(num_positive)
print(num_negative)

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [46]:
num_positive/(num_negative + num_positive)

0.8407087605684476

**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?